In [1]:
using Distributions, LinearAlgebra, Statistics, StatsFuns
cd("../src/")
include("hcmvn.jl")
include("hchol.jl")
include("generate.jl")

Σ_1d_generate (generic function with 1 method)

## Settings

In [2]:
iters = 20
# ns = [512, 1024, 2048]
# ms = [16, 32, 64]
ns = [32, 128, 256]
ms = [16, 32]
d = 4

4

## 4.2 Simulation with a constant covariance matrix

In [3]:
θ = 0.7

0.7

In [4]:
# true value
function cc_prob(n::Int, θ::T, b::Array{T, 1}; ns::Int = 10000) where T<:AbstractFloat
    # calculate values using MC 
    s = 0
    x = rand(Normal(0, 1), ns)
    for i in 1:ns
        integ = 1
        for j in 1:n
            integ *= normcdf((b[j] + sqrt(θ)*x[i])/(1-θ))
        end
        s += integ
    end
    return s/ns
end

cc_prob (generic function with 1 method)

In [ ]:
ans1_m1 = zeros(length(ns), length(ms), iters)
time1_m1 = zeros(length(ns), length(ms))
ans1_m2 = zeros(length(ns), length(ms), iters)
time1_m2 = zeros(length(ns), length(ms))
ans1_m3 = zeros(length(ns), length(ms), iters)
time1_m3 = zeros(length(ns), length(ms))

real = zeros(length(ns), length(ms), iters)

for i in 1:length(ns)
    n = ns[i]
    Σ_const = Σ_const_generate(n, θ)
    for j in 1:length(ms)
        m = ms[j] 
        for p in 1:iters
            a = fill(-Inf, n)
            b = rand(Uniform(0, n), n)
            time1_m1[i, j] += @elapsed ans1_m1[i, j, p] += HMVN(a, b, Σ_const, m)
            time1_m2[i, j] += @elapsed ans1_m2[i, j, p] += HCMVN(a, b, Σ_const, d, m)
            time1_m3[i, j] += @elapsed ans1_m3[i, j, p] += HRCMVN(a, b, Σ_const, d, m)
            real[i, j, p] += cc_prob(n, θ, b)
        end
    end
end

In [ ]:
mapslices(mean, abs.((ans1_m1 .- real) ./real), dims = [3])

In [ ]:
time1_m1 / iters

In [ ]:
mapslices(mean, abs.((ans1_m2 .- real) ./real), dims = [3])

In [ ]:
time1_m2 / iters

In [ ]:
mapslices(mean, abs.((ans1_m3 .- real) ./real), dims = [3])

In [ ]:
time1_m3 / iters

## 4.3 Simulation with 1D exponential covariance matrix

In [ ]:
β = 10

ans2_m1 = zeros(length(ns), length(ms), iters)
time2_m1 = zeros(length(ns), length(ms))
ans2_m2 = zeros(length(ns), length(ms), iters)
time2_m2 = zeros(length(ns), length(ms))
ans2_m3 = zeros(length(ns), length(ms), iters)
time2_m3 = zeros(length(ns), length(ms))

real_1d = zeros(length(ns), length(ms), iters)

for i in 1:length(ns)
    n = ns[i]
    Σ_1d = Σ_1d_generate(n, β)
    L_1d = cholesky(Σ_1d).L
    for j in 1:length(ms)
        m = ms[j] 
        for p in 1:iters
            a = fill(-Inf, n)
            b = rand(Uniform(0, n), n)
            time2_m1[i, j] += @elapsed ans2_m1[i, j, p] += HMVN(a, b, Σ_1d, m)
            time2_m2[i, j] += @elapsed ans2_m2[i, j, p] += HCMVN(a, b, Σ_1d, d, m)
            time2_m3[i, j] += @elapsed ans2_m3[i, j, p] += HRCMVN(a, b, Σ_1d, d, m)
            real_1d[i, j, p] += mvn(L_1d, a, b, ns = 10, N = 1000)
#             real_1d[i, j, p] += mvn(L_1d, a, b, ns = 100, N = 10000)
        end
    end
end

In [ ]:
mapslices(mean, abs.((ans2_m1 .- real_1d) ./real_1d), dims = [3])

In [ ]:
time2_m1 / iters

In [ ]:
mapslices(mean, abs.((ans2_m2 .- real_1d) ./real_1d), dims = [3])

In [ ]:
time2_m2 / iters

In [ ]:
mapslices(mean, abs.((ans2_m3 .- real_1d) ./real_1d), dims = [3])

In [ ]:
time2_m3 / iters